In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime  import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error as mae
import io

energy_demand =pd.read_csv(r"C:\Users\than9\OneDrive - UIU\Smart Grids\Dataset\victoria state dataset.csv")
energy_demand.head()
energy_demand.shape
df=energy_demand.copy()
df['date']=pd.to_datetime(df['date'])
df.rainfall.fillna(df.rainfall.median(), inplace=True)

df.solar_exposure.fillna(df.solar_exposure.median(), inplace=True)
df.info()
df.isna().sum()
#To extract the year
df['Year'] = df['date'].dt.year
#To extract the month
df['Month'] = df['date'].dt.month
#To extract the day
df['Day'] = df['date'].dt.day
df_1=df.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2106 entries, 0 to 2105
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             2106 non-null   datetime64[ns]
 1   demand           2106 non-null   float64       
 2   RRP              2106 non-null   float64       
 3   demand_pos_RRP   2106 non-null   float64       
 4   RRP_positive     2106 non-null   float64       
 5   demand_neg_RRP   2106 non-null   float64       
 6   RRP_negative     2106 non-null   float64       
 7   frac_at_neg_RRP  2106 non-null   float64       
 8   min_temperature  2106 non-null   float64       
 9   max_temperature  2106 non-null   float64       
 10  solar_exposure   2106 non-null   float64       
 11  rainfall         2106 non-null   float64       
 12  school_day       2106 non-null   object        
 13  holiday          2106 non-null   object        
dtypes: datetime64[ns](1), float64(11), objec

In [23]:
norm_scale=MinMaxScaler()
df_scaled=norm_scale.fit_transform(df_1[['RRP','demand','demand_pos_RRP','RRP_positive', 'demand_neg_RRP','RRP_negative', 'frac_at_neg_RRP','min_temperature',
                                         'max_temperature', 'solar_exposure', 'rainfall']])
df_scaled=pd.DataFrame(df_scaled, columns=['RRP','demand','demand_pos_RRP','RRP_positive', 'demand_neg_RRP','RRP_negative', 'frac_at_neg_RRP','min_temperature',
                                         'max_temperature', 'solar_exposure', 'rainfall'])
df_scaled['date']=df_1['date']
df_scaled['holiday']=df_1['holiday']
df_scaled['school_day']=df_1['school_day']
df_scaled=pd.get_dummies(df_scaled)
df_scaled.shape
df_ready=df_scaled.dropna()
df_ready.date.shape
df_ready.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2106 entries, 0 to 2105
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   RRP              2106 non-null   float64       
 1   demand           2106 non-null   float64       
 2   demand_pos_RRP   2106 non-null   float64       
 3   RRP_positive     2106 non-null   float64       
 4   demand_neg_RRP   2106 non-null   float64       
 5   RRP_negative     2106 non-null   float64       
 6   frac_at_neg_RRP  2106 non-null   float64       
 7   min_temperature  2106 non-null   float64       
 8   max_temperature  2106 non-null   float64       
 9   solar_exposure   2106 non-null   float64       
 10  rainfall         2106 non-null   float64       
 11  date             2106 non-null   datetime64[ns]
 12  holiday_N        2106 non-null   bool          
 13  holiday_Y        2106 non-null   bool          
 14  school_day_N     2106 non-null   bool   

In [24]:
df_train = df_ready[df_ready.date<='2019-01-01']
#Test dataset
df_test =  df_ready[df_ready.date>'2019-01-01']
#A reserved copy of the test dataset for plotting
df_resid = df_ready[df_ready.date>'2019-01-01']

#Extract the target feature for training and testing
y_train=df_train["demand"]
y_test=df_test["demand"]

#Extracting the independent features for training and testing
X_train =df_train.drop(labels=['demand', 'date'], axis=1)
X_test=df_test.drop(labels=['demand', 'date'], axis=1)
df_train.shape
print(f'X_train:', len(X_train))
print(f'y_train:', len(y_train))
print(f'X_test:', len(X_test))
print(f'y_tets:', len(y_test))

X_train = X_train.astype('float64')
X_test = X_test.astype('float64')

X_train.info()
X_train.shape
X_train

X_train: 1462
y_train: 1462
X_test: 644
y_tets: 644
<class 'pandas.core.frame.DataFrame'>
Index: 1462 entries, 0 to 1461
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RRP              1462 non-null   float64
 1   demand_pos_RRP   1462 non-null   float64
 2   RRP_positive     1462 non-null   float64
 3   demand_neg_RRP   1462 non-null   float64
 4   RRP_negative     1462 non-null   float64
 5   frac_at_neg_RRP  1462 non-null   float64
 6   min_temperature  1462 non-null   float64
 7   max_temperature  1462 non-null   float64
 8   solar_exposure   1462 non-null   float64
 9   rainfall         1462 non-null   float64
 10  holiday_N        1462 non-null   float64
 11  holiday_Y        1462 non-null   float64
 12  school_day_N     1462 non-null   float64
 13  school_day_Y     1462 non-null   float64
dtypes: float64(14)
memory usage: 171.3 KB


,RRP,demand_pos_RRP,RRP_positive,demand_neg_RRP,RRP_negative,frac_at_neg_RRP,min_temperature,max_temperature,solar_exposure,rainfall,holiday_N,holiday_Y,school_day_N,school_day_Y
0,0.006960,0.430037,0.002832,0.040206,0.978844,0.033333,0.463504,0.518841,0.702454,0.000000,0.0,1.0,1.0,0.0
1,0.008608,0.614724,0.005571,0.147992,0.860295,0.100000,0.540146,0.863768,0.800613,0.000000,1.0,0.0,1.0,0.0
2,0.008921,0.779636,0.004629,0.000000,1.000000,0.000000,0.708029,0.846377,0.791411,0.000000,1.0,0.0,1.0,0.0
3,0.006823,0.484531,0.002521,0.000000,1.000000,0.000000,0.572993,0.359420,0.751534,0.076923,1.0,0.0,1.0,0.0
4,0.007200,0.591797,0.002900,0.000000,1.000000,0.000000,0.525547,0.376812,0.920245,0.000000,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,0.027995,0.713813,0.023785,0.000000,1.000000,0.000000,0.777372,0.794203,0.564417,0.000000,1.0,0.0,1.0,0.0
1458,0.021467,0.515304,0.017230,0.000000,1.000000,0.000000,0.744526,0.431884,0.153374,0.021978,1.0,0.0,1.0,0.0
1459,0.019809,0.447871,0.015564,0.000000,1.000000,0.000000,0.642336,0.397101,0.282209,0.010989,1.0,0.0,1.0,0.0
1460,0.019969,0.477157,0.015725,0.000000,1.000000,0.000000,0.609489,0.420290,0.938650,0.014652,1.0,0.0,1.0,0.0


In [25]:
from keras.preprocessing.sequence import TimeseriesGenerator

sequence_length=24

# Create time series generators for training and testing
train_data_gen = TimeseriesGenerator(X_train.values, y_train.values, length=sequence_length, batch_size=32)
test_data_gen = TimeseriesGenerator(X_test.values, y_test.values, length=sequence_length, batch_size=32)
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Concatenate
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Flatten, Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.layers import GRU
from scipy.optimize import differential_evolution
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, Dropout, LSTM, Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import numpy as np

def huber_loss(y_true, y_pred, delta=1.0):
    error = y_true - y_pred
    is_small_error = K.abs(error) < delta
    small_error_loss = 0.5 * K.square(error)
    large_error_loss = delta * (K.abs(error) - 0.5 * delta)
    return tf.where(is_small_error, small_error_loss, large_error_loss)

class MultiHeadAttentionLayer(Layer):
    def __init__(self, num_heads, key_dim, **kwargs):
        super(MultiHeadAttentionLayer, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.head_dim = key_dim // num_heads

        # Query, Key, and Value Projections
        self.query_projection = Dense(key_dim)
        self.key_projection = Dense(key_dim)
        self.value_projection = Dense(key_dim)

        # Scaled Dot-Product Attention
        self.attention = tf.keras.layers.Attention(use_scale=True)

        # Linear Transformation
        self.output_projection = Dense(key_dim)

    def call(self, inputs):
        # Split inputs into query, key, and value
        query = self.query_projection(inputs)
        key = self.key_projection(inputs)
        value = self.value_projection(inputs)

        # Split into multiple heads
        query_heads = tf.concat(tf.split(query, self.num_heads, axis=-1), axis=0)
        key_heads = tf.concat(tf.split(key, self.num_heads, axis=-1), axis=0)
        value_heads = tf.concat(tf.split(value, self.num_heads, axis=-1), axis=0)

        # Scaled Dot-Product Attention
        attention_output = self.attention([query_heads, key_heads, value_heads])

        # Concatenate heads
        attention_output = tf.concat(tf.split(attention_output, self.num_heads, axis=0), axis=-1)

        # Linear Transformation
        output = self.output_projection(attention_output)

        return output
    
    def get_config(self):
        config = super(MultiHeadAttentionLayer, self).get_config()
        config.update({'num_heads': self.num_heads, 'key_dim': self.key_dim})
        return config



class TabularLSTMGRUModel:
    def __init__(self, input_shape, lstm_units=[64, 32], gru_units=[64, 32], output_units=1, num_heads=8):
        self.input_shape = input_shape
        self.lstm_units = lstm_units
        self.gru_units = gru_units
        self.output_units = output_units
        self.num_heads = num_heads
        self.model = self.build_model()

    def build_model(self):
        model = tf.keras.Sequential()

        # Convolutional layers
        model.add(Conv1D(filters=256, kernel_size=2, activation='relu', input_shape=self.input_shape))
        model.add(MaxPooling1D(pool_size=2))
        model.add(tf.keras.layers.BatchNormalization())

        model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(tf.keras.layers.BatchNormalization())

        model.add(MultiHeadAttentionLayer(num_heads=self.num_heads, key_dim=self.lstm_units[-1]))

        model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(tf.keras.layers.BatchNormalization())

        # Multi-Head Attention layer
        model.add(MultiHeadAttentionLayer(num_heads=self.num_heads, key_dim=self.lstm_units[-1]))

        # GRU layers
        for units in self.gru_units:
            model.add(GRU(units, return_sequences=True, activation='relu'))

        model.add(Flatten())
        model.add(Dense(self.output_units))

        return model

    def compile(self, learning_rate=0.001):
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        self.model.compile(loss=huber_loss, optimizer=optimizer)

    def fit(self, train_data_gen, epochs=10):
        self.model.fit(train_data_gen, epochs=epochs)

    def evaluate(self, test_data_gen):
        return self.model.evaluate(test_data_gen)

    def predict(self, data_gen):
        return self.model.predict(data_gen)

    def summary(self):
        return self.model.summary()
# Assuming you have the input_shape and train_data_gen defined
input_shape = (24, 14)
lstm_units = [64, 32, 16, 8]
gru_units = [64, 32, 16, 8]  # Define the units for each GRU layer
lstm_gru_model = TabularLSTMGRUModel(input_shape, lstm_units=lstm_units, gru_units=gru_units)
num_epochs = 100
# Compile the model
lstm_gru_model.compile()
lstm_gru_model.summary()
# Fit the model to the training data
lstm_gru_model.fit(train_data_gen, epochs=num_epochs)

# Evaluate the model on your test data (assuming you have test_data_gen)
test_loss = lstm_gru_model.evaluate(test_data_gen)
print(f"Test Loss: {test_loss}")

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score, max_error
import numpy as np

y_pred = lstm_gru_model.predict(test_data_gen)
y_true = y_test.values[:len(y_pred)].flatten()

# Calculate metrics
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
r2 = r2_score(y_true, y_pred)
explained_variance = explained_variance_score(y_true, y_pred)
max_err = max_error(y_true, y_pred)

# Additional metrics for poisson and gamma distributions (you may need to replace these with your actual predictions)
poisson_deviance = np.mean(2 * (y_true * np.log(y_true / y_pred) - (y_true - y_pred)))
gamma_deviance = np.mean(2 * ((np.log(y_pred) - np.log(y_true)) + (y_true - y_pred) / y_pred))

# Print or use the calculated metrics as needed
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("MAPE:", mape)
print("Explained Variance:", explained_variance)
print("Max Error:", max_err)
print("Mean Poisson Deviance:", poisson_deviance)
print("Mean Gamma Deviance:", gamma_deviance)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 23, 256)           7424      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 11, 256)          0         
 1D)                                                             
                                                                 
 batch_normalization_3 (Batc  (None, 11, 256)          1024      
 hNormalization)                                                 
                                                                 
 conv1d_4 (Conv1D)           (None, 10, 128)           65664     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 5, 128)           0         
 1D)                                                             
                                                      

Federated Learning

In [26]:
import numpy as np

NUM_CLIENTS = 10
NUM_ROUNDS = 3  # You can adjust this value based on the number of clients you want

clients = []

for i in range(NUM_CLIENTS):
    client_data = X_train.iloc[i * (len(X_train) // NUM_CLIENTS):(i + 1) * (len(X_train) // NUM_CLIENTS)]
    client_labels = y_train.iloc[i * (len(y_train) // NUM_CLIENTS):(i + 1) * (len(y_train) // NUM_CLIENTS)]
    clients.append((client_data, client_labels))
client_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, 1314 to 1459
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RRP              146 non-null    float64
 1   demand_pos_RRP   146 non-null    float64
 2   RRP_positive     146 non-null    float64
 3   demand_neg_RRP   146 non-null    float64
 4   RRP_negative     146 non-null    float64
 5   frac_at_neg_RRP  146 non-null    float64
 6   min_temperature  146 non-null    float64
 7   max_temperature  146 non-null    float64
 8   solar_exposure   146 non-null    float64
 9   rainfall         146 non-null    float64
 10  holiday_N        146 non-null    float64
 11  holiday_Y        146 non-null    float64
 12  school_day_N     146 non-null    float64
 13  school_day_Y     146 non-null    float64
dtypes: float64(14)
memory usage: 17.1 KB


In [27]:
for i in range(NUM_CLIENTS):
    test_data = X_test.iloc[i * (len(X_test) // NUM_CLIENTS):(i + 1) * (len(X_test) // NUM_CLIENTS)]
    test_labels = y_test.iloc[i * (len(y_test) // NUM_CLIENTS):(i + 1) * (len(y_test) // NUM_CLIENTS)]

In [30]:
import time
import tensorflow as tf
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
start_time = time.time()
# Constants
NUM_ROUNDS = 3  # You can adjust the number of rounds as needed
NUM_CLIENTS = 10  # Assuming clients is the list of available clients
sequence_length = 24
max_epoch=20
test_losses = []
# Pruning parameters
layer_preserving_rate = 0.7  
pruning_threshold = 0.1  
convergence_curve_method1 = []
# Federated learning loop
for round_num in range(NUM_ROUNDS):

    # Select clients
    selected_client_indices = np.random.choice(len(clients), size=int(NUM_CLIENTS * 0.5), replace=False)
    selected_clients = [clients[i] for i in selected_client_indices]

    # Transmit the demand forecasting model to the selected clients
    for client_data, client_labels in selected_clients:
        # Convert non-numeric columns to numeric values

        # Clone the model and set weights
        client_model = tf.keras.models.clone_model(lstm_gru_model.model)
        client_model.set_weights(lstm_gru_model.model.get_weights())

        # Compile the client model
        client_model.compile(optimizer=Adam(learning_rate=0.001), loss=huber_loss)  # Use Huber loss

        steps_per_epoch = 3  # Corrected the steps_per_epoch calculation

        # Layer-wise pruning implementation
        for t in range(1, max_epoch + 1):
            # Update local model
            client_model.fit(
                TimeseriesGenerator(client_data.values, client_labels.values,
                                length=sequence_length, batch_size=32),
                epochs=20,
                steps_per_epoch=steps_per_epoch
            )

            # Generate layer-wise pruning indicators
            layer_preserving_indicators = np.random.binomial(1, layer_preserving_rate, size=len(client_model.layers))
            pruned_local_model = tf.keras.models.clone_model(lstm_gru_model.model)
            pruned_local_model.set_weights(client_model.get_weights())
            for layer_index, preserve_indicator in enumerate(layer_preserving_indicators):
                if not preserve_indicator:
                    pruned_local_model.layers[layer_index].set_weights([np.zeros_like(w) for w in pruned_local_model.layers[layer_index].get_weights()])

            # Magnitude-based pruning
            for layer in pruned_local_model.layers:
                if hasattr(layer, 'get_weights'):
                    weights = layer.get_weights()
                    pruned_weights = [np.where(np.abs(w) > pruning_threshold, w, 0.0) for w in weights]
                    layer.set_weights(pruned_weights)

            # In a real federated learning scenario, upload the pruned local model to the parameter server

        aggregated_weights = []
        for layer_index in range(len(lstm_gru_model.model.get_weights())):
            layer_weights = np.mean([pruned_local_model.get_weights()[layer_index], lstm_gru_model.model.get_weights()[layer_index]], axis=0)
            aggregated_weights.append(layer_weights)
            
        # Update global model
        lstm_gru_model.model.set_weights(aggregated_weights)

    # Evaluate the global model after local training
    test_datagen = TimeseriesGenerator(test_data.values, test_labels.values,
                                length=sequence_length, batch_size=32)
    test_loss = lstm_gru_model.evaluate(test_datagen)
    print('Round {}: test_loss = {}'.format(round_num, test_loss))
    test_losses.append(test_loss)
    convergence_curve_method1.append(test_loss)

end_time = time.time()
total_time = end_time - start_time
# Calculate minimum test loss
min_test_loss = np.min(test_losses)
print('Minimum Test Loss: {}'.format(min_test_loss))

# Calculate mean test loss
mean_test_loss = np.mean(test_losses)
print('Mean Test Loss: {}'.format(mean_test_loss))

print(f"Total execution time: {total_time} seconds")
lstm_gru_model.summary()

Epoch 1/20
3/3 [==============================] - 6s 82ms/step - loss: 0.1177
Epoch 2/20
3/3 [==============================] - 0s 65ms/step - loss: 0.1332
Epoch 3/20
3/3 [==============================] - 0s 73ms/step - loss: 0.0964
Epoch 4/20
3/3 [==============================] - 0s 58ms/step - loss: 0.1036
Epoch 5/20
3/3 [==============================] - 0s 62ms/step - loss: 0.1007
Epoch 6/20
3/3 [==============================] - 0s 63ms/step - loss: 0.0880
Epoch 7/20
3/3 [==============================] - 0s 59ms/step - loss: 0.0941
Epoch 8/20
3/3 [==============================] - 0s 55ms/step - loss: 0.0952
Epoch 9/20
3/3 [==============================] - 0s 78ms/step - loss: 0.0840
Epoch 10/20
3/3 [==============================] - 0s 84ms/step - loss: 0.0961
Epoch 11/20
3/3 [==============================] - 0s 77ms/step - loss: 0.0574
Epoch 12/20
3/3 [==============================] - 0s 61ms/step - loss: 0.0493
Epoch 13/20
3/3 [==============================] - 0s 61ms/st

In [31]:
import time
import tensorflow as tf
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
start_time = time.time()
# Constants
NUM_ROUNDS = 3  
test_losses = []
# You can adjust the number of rounds as needed
NUM_CLIENTS = 10  # Assuming clients is the list of available clients
sequence_length = 24
convergence_curve_method2 = []
# Federated learning loop
for round_num in range(NUM_ROUNDS):

    # Select clients
    selected_client_indices = np.random.choice(len(clients), size=int(NUM_CLIENTS * 0.5), replace=False)
    selected_clients = [clients[i] for i in selected_client_indices]

    # Transmit the demand forecasting model to the selected clients
    for client_data, client_labels in selected_clients:
        # Convert non-numeric columns to numeric values

        # Clone the model and set weights
        client_model = tf.keras.models.clone_model(lstm_gru_model.model)
        client_model.set_weights(lstm_gru_model.model.get_weights())

        # Compile the client model
        client_model.compile(optimizer=Adam(learning_rate=0.001), loss=huber_loss)

        steps_per_epoch = len(client_data) // 64

        client_model.fit(
            TimeseriesGenerator(client_data.values, client_labels.values,
                                length=sequence_length, batch_size=32),
            epochs=20,
            steps_per_epoch=steps_per_epoch
        )

        # Aggregate the model
        new_weights = []
        for layer_index in range(len(lstm_gru_model.model.get_weights())):
            new_layer_weights = np.mean(
                [client_model.get_weights()[layer_index], lstm_gru_model.model.get_weights()[layer_index]], axis=0
            )
            new_weights.append(new_layer_weights)
        lstm_gru_model.model.set_weights(new_weights)


    # Evaluate the global model after local training
    test_datagen=TimeseriesGenerator(test_data.values, test_labels.values,
                                length=sequence_length, batch_size=32)
    test_loss = lstm_gru_model.evaluate(test_datagen)
    print('Round {}: test_loss = {}'.format(round_num, test_loss))
    test_losses.append(test_loss)
    convergence_curve_method2.append(test_loss)
    
end_time = time.time()
total_time = end_time - start_time
# Calculate minimum test loss
min_test_loss = np.min(test_losses)
print('Minimum Test Loss: {}'.format(min_test_loss))

# Calculate mean test loss
mean_test_loss = np.mean(test_losses)
print('Mean Test Loss: {}'.format(mean_test_loss))
print(f"Total execution time: {total_time} seconds")
lstm_gru_model.summary()

Epoch 1/20
3/3 [==============================] - 6s 63ms/step - loss: 0.0590
Epoch 2/20
3/3 [==============================] - 0s 43ms/step - loss: 0.0566
Epoch 3/20
3/3 [==============================] - 0s 47ms/step - loss: 0.0544
Epoch 4/20
3/3 [==============================] - 0s 39ms/step - loss: 0.0520
Epoch 5/20
3/3 [==============================] - 0s 55ms/step - loss: 0.0603
Epoch 6/20
3/3 [==============================] - 0s 44ms/step - loss: 0.0589
Epoch 7/20
3/3 [==============================] - 0s 47ms/step - loss: 0.0588
Epoch 8/20
3/3 [==============================] - 0s 55ms/step - loss: 0.0510
Epoch 9/20
3/3 [==============================] - 0s 50ms/step - loss: 0.0518
Epoch 10/20
3/3 [==============================] - 0s 39ms/step - loss: 0.0350
Epoch 11/20
3/3 [==============================] - 0s 47ms/step - loss: 0.0399
Epoch 12/20
3/3 [==============================] - 0s 39ms/step - loss: 0.0278
Epoch 13/20
3/3 [==============================] - 0s 47ms/st